
![alt text](um6p.png)

***Abderahmane HAMDOUCHI***



**Master SDAD**

**UM6P**

\newpage

# Objectif du Tp

Ce tp a pour objectif de se familiariser avec le framework Spark et d'implémenter les  algorithmes de recommondation en particuler et ALS en particulier sur des données movies pour recommender à l'utilisateur un film , et on appliquer des hyper paramètres pour retenir à la fin le meilleur algorithme mais tous les dirivé de ALS ont donné le meme score RMSE , je crois ça revient à la petite quantité de nos données 

\newpage

\tableofcontents 

\newpage

# Système de recommendation en utilisant SPARK

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('rec').getOrCreate()

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

La bibliothèque Spark MLlib pour Machine Learning fournit une implémentation de filtrage collaboratif à l'aide des moindres carrés alternés (Alternating Least Squares). L'implémentation dans MLlib a ces paramètres:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

## Importer le fichier csv 

In [0]:
data = spark.read.csv('/FileStore/tables/movielens_ratings.csv',inferSchema=True,header=True)

In [0]:
data.head()

Out[36]: Row(movieId=2, rating=3.0, userId=0)

In [0]:
data.describe().toPandas()

Out[39]:

,summary,movieId,rating,userId
0,count,1501,1501,1501
1,mean,49.40572951365756,1.7741505662891406,14.383744170552964
2,stddev,28.937034065088994,1.187276166124803,8.591040424293272
3,min,0,1.0,0
4,max,99,5.0,29


Nous pouvons faire une séparation pour évaluer les performances de notre modèle, mais gardez à l'esprit qu'il est très difficile de savoir de manière concluante à quel point un système de recommandation fonctionne vraiment pour certains sujets. Surtout si la subjectivité est impliquée, par exemple, tout le monde qui aime Star Wars ne va pas aimer le Star Trek, même si un système de recommandation peut suggérer le contraire.

In [0]:
(training, test) = data.randomSplit([0.8, 0.2])

## Implémenter le fichier ALS

In [0]:
# Construire une recommendation en utilisat ALS
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

## Appliquer les hyper paramètres sur l'algorithme ALS pour avoir le meilleur parmi eux

In [0]:
import numpy as np 
regParam_set= np.arange(0.01,0.1,0.01)
model_set=[]
for regP in regParam_set:
  als_temp = ALS(maxIter=10, regParam=regP, userCol="userId", itemCol="movieId", ratingCol="rating")
  model_set.append(als.fit(training))
  

In [0]:
predection_set=[]
for mod in model_set:
  predection_set.append(mod.transform(test))
  
  

In [0]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
for pred in predection_set:
  rmse_temp = evaluator.evaluate(pred)
  print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.8427636187829386
Root-mean-square error = 1.8427636187829386
Root-mean-square error = 1.8427636187829386
Root-mean-square error = 1.8427636187829386
Root-mean-square error = 1.8427636187829386
Root-mean-square error = 1.8427636187829386
Root-mean-square error = 1.8427636187829386
Root-mean-square error = 1.8427636187829386
Root-mean-square error = 1.8427636187829386

Après avoir appliquer des hyper paramètres sur l'algorithme ALS  avec un nombre d'itération et le paramètre  regParam avec un intervalle entre (0.01 et 0.01 avec un pas de 0.01) et on a augmenter le nobre d'itération à 10 mais on a eu le meme rmse qui est le meilleur obtenu dans notre cas d'étude et par ce que la data est trop petite pour cela la fonction cout ne s'améliore pas davantage

***Alors on garde le modèle prédifini du TP car il a les memes perfermonces***

Now let's see hwo the model performed!

In [0]:
predictions = model.transform(test)

In [0]:
predictions.toPandas()

Out[42]:

,movieId,rating,userId,prediction
0,31,1.0,19,3.347229
1,31,1.0,24,0.948655
2,31,2.0,25,-0.064031
3,31,1.0,27,2.128186
4,85,1.0,5,0.826860
...,...,...,...,...
290,36,1.0,14,0.971322
291,36,1.0,18,3.960662
292,36,4.0,26,4.670935
293,36,2.0,29,2.103971


In [0]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.8427636187829386

Alors maintenant que nous avons le modèle, comment pourriez-vous réellement fournir une recommandation à un utilisateur?

De la même manière que nous l'avons fait avec les données de test! Par exemple:

In [0]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [0]:
# l'utilisateur 11 à 10 ratings dans les données de test 
single_user.toPandas()

Out[43]:

,movieId,userId
0,18,11
1,19,11
2,23,11
3,30,11
4,35,11
5,82,11


In [0]:
reccomendations = model.transform(single_user)

In [0]:
reccomendations.orderBy('prediction',ascending=False).toPandas()

Out[44]:

,movieId,userId,prediction
0,30,11,3.202430
1,82,11,0.722134
2,23,11,0.437919
3,35,11,0.414738
4,19,11,-0.387644
5,18,11,-3.904463
